In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from sklearn.model_selection import train_test_split
import os

In [2]:
# Define the folder name
folder_name = 'Predictions'

# Specify the path where you want to create the folder
folder_path = '/content/' + folder_name

# Create the folder
os.makedirs(folder_path, exist_ok=True)

In [3]:
!gdown 'https://drive.google.com/uc?id=1mrN0MYL9uwjzw_jZmvlb4-lCnmBM3b7A'

Downloading...
From: https://drive.google.com/uc?id=1mrN0MYL9uwjzw_jZmvlb4-lCnmBM3b7A
To: /content/Forest.zip
100% 181M/181M [00:01<00:00, 166MB/s]


In [4]:
!unzip '/content/Forest.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_73.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_77.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_78.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_80.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_81.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_87.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/119079_mask_88.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_mask_00.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_mask_01.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_mask_03.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_mask_04.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_mask_11.jpg  
  inflating: Forest Segmented/Forest Segmented/masks/122104_m

In [5]:
# Importing the required libraries
import os
import shutil

# Define the path of the input directory
input_dir_path = '/content/Forest Segmented/Forest Segmented/images'  # Replace with your directory path

# Define the paths for the output directories
output_dir1_path = '/content/Forest Segmented/Forest Segmented/train_images'
output_dir2_path = '/content/Forest Segmented/Forest Segmented/test_images'


# Create the output directories
os.makedirs(output_dir1_path, exist_ok=True)
os.makedirs(output_dir2_path, exist_ok=True)

# Get the list of files in the input directory
files = os.listdir(input_dir_path)

# Sort the files to maintain consistency across runs
files.sort()

# Split the files into two parts
part1 = files[0:49]
part2 = files[50:19]

# Move the files to the output directories
for file in part1:
    src_path = os.path.join(input_dir_path, file)
    dst_path = os.path.join(output_dir1_path, file)
    shutil.move(src_path, dst_path)

for file in part2:
    src_path = os.path.join(input_dir_path, file)
    dst_path = os.path.join(output_dir2_path, file)
    shutil.move(src_path, dst_path)

In [6]:
# Define the path of the input directory
input_dir_path = '/content/Forest Segmented/Forest Segmented/masks'  # Replace with your directory path

# Define the paths for the output directories
output_dir1_path = '/content/Forest Segmented/Forest Segmented/train_masks'
output_dir2_path = '/content/Forest Segmented/Forest Segmented/test_masks'


# Create the output directories
os.makedirs(output_dir1_path, exist_ok=True)
os.makedirs(output_dir2_path, exist_ok=True)

# Get the list of files in the input directory
files = os.listdir(input_dir_path)

# Sort the files to maintain consistency across runs
files.sort()

# Split the files into two parts
part1 = files[0:49]
part2 = files[50:59]

# Move the files to the output directories
for file in part1:
    src_path = os.path.join(input_dir_path, file)
    dst_path = os.path.join(output_dir1_path, file)
    shutil.move(src_path, dst_path)

for file in part2:
    src_path = os.path.join(input_dir_path, file)
    dst_path = os.path.join(output_dir2_path, file)
    shutil.move(src_path, dst_path)

In [7]:
#creating a class to do the successive colvolutions
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(      #creates a sequence of convolution operations 
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),    #kernel_size, stride, padding : results in same convolution, no bias (b) term
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward (self, x):
    return self.conv(x)

In [8]:
class UNET(nn.Module):

  def __init__(self, in_channels = 3, out_channels = 1, features = [64, 128, 256, 512]):    #in_channels = 3 as RGB, features: number of channels in successive layers
    super(UNET, self).__init__()
    self.ups = nn.ModuleList()    #special list which stores instances of nn.Module
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

    #down part of UNET
    for feature in features:
      self.downs.append (DoubleConv(in_channels, feature))
      in_channels = feature

    #up part of UNET
    for feature in reversed(features):
      self.ups.append (nn.ConvTranspose2d(feature*2, feature, kernel_size = 2, stride = 2))
      self.ups.append (DoubleConv (feature*2 , feature))

    #bottommost part
    self.bottleneck = DoubleConv (features[-1], features[-1]*2)

    #last layer
    self.final_conv = nn.Conv2d (features[0], out_channels, kernel_size = 1)    

  
  def forward (self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool (x)

    x = self.bottleneck (x)
    skip_connections = skip_connections[::-1]

    #doing Transposed convolution then double conv repeatedly
    for idx in range (0, len(self.ups), 2):     #self.ups contains the 2dconv as well as Transpose2D tesnors
      x = self.ups[idx](x)    #contains the Transposed Convolution function
      skip_connection = skip_connections [idx//2]

      if x.shape != skip_connection.shape:    #happens if skip connection has odd size, x would have 1 pixel less as pooling floors
        x = TF.resize (x, size = skip_connection.shape[2:])   #skip batch size, number of channels

      concat_skip = torch.cat ((skip_connection, x), dim = 1)   #join along 1 dimension (vertical join)
      x =  self.ups[idx+1](concat_skip)   #contains the double conv

    return self.final_conv(x)

In [9]:
#Dataset loading
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

In [10]:
class ImageDataset(Dataset):
  def __init__(self, image_dir, mask_dir, transform=None):    #transform = None as a standard
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform 
    self.images = os.listdir(image_dir)

  def __len__(self):    #dunder methods (double underscore) - invoked automatically when an instance created
    return (len(self.images))
  
  def __getitem__(self, index):
    img_path = os.path.join (self.image_dir, self.images[index])
    path1 = self.images[index]
    path2 = path1[-7::1]
    path3 = path1[-11::-1]
    path4 = path3[-1::-1]
    path1 = path4 + 'mask' + path2
    mask_path = os.path.join (self.mask_dir, path1)
    image = np.array(Image.open(img_path).convert("RGB"))
    #mask_path = os.path.join (self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))

    if os.path.exists(mask_path):
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0
    else:
        # If the mask file is missing, skip this element or handle it as desired
        # In this example, we assign a default mask
        mask = np.zeros_like(image[:, :, 0], dtype=np.float32)    
    
    mask = np.array(Image.open(mask_path).convert("L"), dtype = np.float32)
    mask[mask == 255.0 ] = 1.0

    if self.transform is not None:
      augmentations = self.transform(image = image, mask = mask)
      image = augmentations ['image']
      mask = augmentations ['mask']
    
    return image, mask

In [11]:
#training
import albumentations as A
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader

In [12]:
#Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240 
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = '/content/Forest Segmented/Forest Segmented/train_images'
TRAIN_MASK_DIR = '/content/Forest Segmented/Forest Segmented/train_masks'
VAL_IMG_DIR = '/content/Forest Segmented/Forest Segmented/test_images'
VAL_MASK_DIR = '/content/Forest Segmented/Forest Segmented/test_masks'

In [13]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
  loop = tqdm (loader)
  

  for batch_idx, (data, targets) in enumerate(loop):
    data = data.to(device = DEVICE)
    targets = targets.float().unsqueeze(1).to(device = DEVICE)

    #forward
    with torch.cuda.amp.autocast():
      predictions = model(data)
      loss = loss_fn (predictions, targets)

    #backward
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    #tqdm loop
    loop.set_postfix(loss = loss.item())

In [28]:
def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir, 
    batch_size,
    train_transform,
    val_transform,
    num_workers = 4,
    pin_memory=True
):
  train_ds = ImageDataset(
      image_dir = train_dir,
      mask_dir = train_maskdir,
      transform = train_transform,
  )

  train_loader = DataLoader(
      train_ds,
      batch_size = batch_size,
      num_workers = num_workers,
      pin_memory = pin_memory,
      shuffle = True,
  )

  val_ds = ImageDataset(
      image_dir = val_dir,
      mask_dir = val_maskdir,
      transform = val_transform,
  )

  val_loader = DataLoader(
    val_ds,
    batch_size = batch_size,
    num_workers = num_workers,
    pin_memory = pin_memory,
    shuffle = False,
  )

  return train_loader, val_loader


def check_accuracy (loader, model, device = 'cuda'):
  num_correct = 0
  num_pixels = 0
  dice_score = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)
      preds = torch.sigmoid (model(x))
      preds = (preds > 0.5).float()
      num_correct += (preds == y).sum()
      num_pixels += torch.numel(preds)
      num_pixels = num_pixels 
      dice_score = (2 * (preds * y).sum() / ((preds + y).sum()))
  
  print (f'Got {num_correct}/{num_pixels}')
  #with accuracy {num_correct/num_pixels*100:.2f}
  print (f'Dice score: {dice_score/(len(loader)+ 1)}')

  model.train()

def save_predictions_as_imgs (loader, model, folder = '/content/sample_data', device = 'cuda'):
  model.eval()
  for idx, (x, y) in enumerate (loader):
    x = x.to (device = device)
    with torch.no_grad():
      preds = torch.sigmoid(model(x))
      preds = (preds > 0.5).float()
      torchvision.utils.save_image(preds, f"{folder}/pred_{idx}.png")
      torchvision.utils.save_image(y.unsqueeze(1), f"{folder}/label_{idx}.png")
    
  model.train()

In [15]:
def save_checkpoint (state, filename = 'my_checkpoint.pth.tar'):
  print("=> Saving Checkpoint")
  torch.save(state, filename)

In [16]:
def main():
  train_transform = A.Compose(
      [A.Resize (height = IMAGE_HEIGHT, width = IMAGE_WIDTH),
       A.Rotate (limit=35, p=1.0),
       A.HorizontalFlip (p=0.5),
       A.Normalize(
          mean = [0.0, 0.0, 0.0],
          std = [1.0, 1.0, 1.0],
          max_pixel_value = 255.0,
       ),
       ToTensorV2(),
       ],
  )

  val_transforms = A.Compose(
      [A.Resize (height = IMAGE_HEIGHT, width = IMAGE_WIDTH),
       A.Normalize(
          mean = [0.0, 0.0, 0.0],
          std = [1.0, 1.0, 1.0],
          max_pixel_value = 255.0,
       ),
       ToTensorV2(),
       ],
  )

  model = UNET (in_channels = 3, out_channels = 1).to(DEVICE)
  loss_fn = nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

  train_loader, val_loader = get_loaders(
      TRAIN_IMG_DIR,
      TRAIN_MASK_DIR,
      VAL_IMG_DIR,
      VAL_MASK_DIR,
      BATCH_SIZE,
      train_transform,
      val_transforms,
      NUM_WORKERS,
      PIN_MEMORY
  )

  scaler = torch.cuda.amp.GradScaler()

  for epoch in range(NUM_EPOCHS):
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    checkpoint =  {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint)

    check_accuracy(val_loader, model, device = DEVICE) 

    save_predictions_as_imgs(val_loader, model, folder = '/content/Predictions', device = DEVICE)

In [30]:
if __name__ == '__main__':
  main()

100%|██████████| 7/7 [02:57<00:00, 25.35s/it, loss=0.516]


=> Saving Checkpoint
Got 0/0
Dice score: 0.0


100%|██████████| 7/7 [03:04<00:00, 26.32s/it, loss=-.383]


=> Saving Checkpoint
Got 0/0
Dice score: 0.0


100%|██████████| 7/7 [03:01<00:00, 25.88s/it, loss=0.682]


=> Saving Checkpoint
Got 0/0
Dice score: 0.0
